In [95]:
from sklearn.datasets import load_iris
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import cv2 
from PIL import Image
import glob
from skimage import feature
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import pickle
import pywt
import re
import copy
import random
import time


import warnings
warnings.filterwarnings('ignore')

In [46]:
def detect_contour(image):
    obj_dict = {}
    retval, bw = cv2.threshold(image, 50, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    contours, hierarchy = cv2.findContours(bw, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    detect_count = 0

    for i in range(0, len(contours)):
        area = cv2.contourArea(contours[i])
        
        if area < 1e2 or 1e5 < area:
            continue
            
        if len(contours[i]) > 0:
            rect = contours[i]
            x, y, w, h = cv2.boundingRect(rect)
            
            angle_data = cv2.minAreaRect(contours[i])
            angle = angle_data[2]
            point = [y, y+h, x, x+w, angle]
            w_h = w + h
            obj_dict[w_h] = point 
    return obj_dict[max(obj_dict)]

In [96]:
def burst_feature_point(color_paths, depth_paths, mode):
    mode_list = ['train', 'test']
    if mode not in mode_list:
        raise Exception('Error!')
    train_feature = []
    hog_list = []
    name_list = []
    X_train = []
    
    for color_path, depth_path in zip(color_paths, depth_paths):
        depth_image = cv2.imread(depth_path, 0)
        color_image = cv2.imread(color_path, 0)
        
        # y_trainの作成
        slash_index = color_path[13:].find('/')
        name_list.append(color_path[13:13+slash_index])
        
        image_list = []
        depth_copy = copy.deepcopy(depth_image)
        color_copy = copy.deepcopy(color_image)

        for col in depth_image:
            for one in col:
                if one > 1000:
                    image_list.append(0)
                else:
                    image_list.append(one)

        for y, col in enumerate(depth_image):
            for x, pixel in enumerate(col):
                if pixel > 0 and pixel < 390: 
                    depth_image[y][x] = 255
                else:
                    depth_image[y][x] = 0
        image = np.array(depth_image, dtype=np.uint8)

        y1, y2, x1, x2, angle = detect_contour(image)
        img = color_image[y1:y2, x1:x2]
        depth_cut = depth_copy[y1:y2, x1:x2]
        
        # 特徴点の場所をカット
        # 特徴量を作成
        width, height = img.shape
        q2_x = int(width/2)
        q2_y = int(height/2)
        q1_x = int(q2_x/2)
        q1_y = int(q2_y/2)
        q3_x = int(q2_x+q1_x)
        q3_y = int(q2_y+q1_y)
        b_x = 16
        b_y = 16

        x_axis = [q1_x, q1_x, q3_x]
        y_axis = [q1_y, q2_y, q3_y]

        cut_img_list = []
        features_list = []
        hog_img_list = []
        for x_q in x_axis:
            for y_q in y_axis:
                cut_img = img[x_q-b_x:x_q+b_x, y_q-b_y:y_q+b_y]
                features, hog_img = feature.hog(cut_img, orientations=8, pixels_per_cell=(8, 8),
                                                cells_per_block=(1, 1), visualize=True, multichannel=False)
                features_list.append(features)
                hog_img_list.append(hog_img)

# バウンディングボックスの場所の可視化
#         for x_q in x_axis:
#             for y_q in y_axis: 
#                 x1 = x_q-16
#                 x2 = x_q+16
#                 y1 = y_q-16
#                 y2 = y_q+16
#                 cv2.rectangle(img, (y1, x1), (y2, x2), color=(0, 255, 0), thickness=2)
#         save_path = 'boundingbox' + path[path.find('/'):]
#         cv2.imwrite(save_path, img)
        
        train_feature.append(features_list)
        hog_list.append(hog_img_list)
        
# 入力毎に特徴量を1次元に格納
    for col_feature in train_feature:
        data = np.array([])
        data = np.append(data, col_feature)
        X_train.append(data)
    X_train = np.array(X_train)    
    
    if mode == 'train':
        y_dict = {}
        le = LabelEncoder()
        y_train = le.fit_transform(name_list)
        decoded = le.inverse_transform(y_train)

        for name, name_id in zip(decoded, y_train):
            y_dict[name_id] = name
        return X_train, y_train, y_dict
    else:
        return X_train

In [92]:
train_path = glob.glob('image_folder/*zoom*')
test_path = glob.glob('test_image/*')

In [93]:
color_path = glob.glob('image_folder/*/*color*')
depth_path = []

random.seed(0)
random.shuffle(color_path)
for path in color_path:
    new_path = path.replace('color', 'depth')
    depth_path.append(new_path)
    
train_color = color_path[:70]
train_depth = depth_path[:70]

test_color = color_path[70:]
test_depth = depth_path[70:]

In [94]:
X_train, y_train, y_dict = burst_feature_point(train_color, train_depth, 'train')
X_test = burst_feature_point(test_color, test_depth, 'test')

start
start
start
start
start
start
start
start
start
start
start
start
start
start
start


KeyboardInterrupt: 

In [87]:
X_train.shape

(70, 1152)

In [88]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [89]:
y_pred = knn.predict(X_test)
for index in y_pred:
    print(y_dict[index])

iris
curry
curry
curry
curry


In [90]:
test_color

['image_folder/iris/iris_color_1.jpg',
 'image_folder/apple/apple_color_0_221.jpg',
 'image_folder/curry/curry_color_0_868.jpg',
 'image_folder/kurumiA/kurumiA_color_0.jpg',
 'image_folder/kurumiA/kurumiA_color_0_346.jpg']